In [1]:
# %pip install statsmodels
# %pip install mlxtend
# %pip install xgboost

In [2]:
# Import necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split
import itertools
import random
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import xgboost as xgb
from sklearn.model_selection import train_test_split
import random
import pandas as pd

In [13]:

df_All_1 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_1o2.csv')
df_All_2 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_2o2.csv')
# Concatenate (union) the dataframes
df_All = pd.concat([df_All_1, df_All_2], ignore_index=True)

print(df_All.head())

df_TagDesc = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\CSV\Not for Processing\TagDesc.csv')


    425 %Al    Butanol   Decanol    Ethanol   Hexanol   Octanol   AYC55580  \
0  6.319560  30.366200  1.030260  88.289900  1.365840  1.492120  18.297300   
1  6.319915  30.678500  1.063260  89.685450  1.397135  1.548230  18.040650   
2  6.328340  32.399333  1.647950  88.677200  1.994413  2.454430  16.637633   
3  6.336760  32.845950  1.928805  87.080875  2.288335  2.897523  15.779400   
4  6.337460  33.387025  2.352595  84.489175  2.734682  3.571185  15.199680   

    DI55102   DI55152   DI55580      FC42428      FC52018    FC55003  \
0  0.963347  0.924879  1.017340  40773.90000  42583.40000  5622.8300   
1  0.965043  0.924515  1.014885  40816.90000  44547.30000  5308.8850   
2  0.968107  0.929955  1.015593  41080.96667  45105.73333  5489.7400   
3  0.969573  0.932388  1.014703  41606.60000  45413.77500  5709.1825   
4  0.971022  0.934531  1.014094  41722.12000  45547.02000  5653.1640   

       FC55009      FC55102      FC55152   FC55552      FC55555      FC55569  \
0  1770.180000  54

In [4]:
# Set max columns to display
pd.set_option('display.max_columns', None)

In [5]:
# df_All = df_All[df_All['Date'] > '2022-06-15 00:00:00']

In [6]:
selected_columns = ['Butanol', 
                    'FC55569',
                    'DI55152',
                    '425 %Al',
                    'TC55552',
                    'FC55003',
                    'LC55555',
                    'FFC55553',
                    'FFC55555',
                    'TC55555',
                    'TI55021',
                    'PI55004',
                    'FC55552'
                    ]
existing_columns = [col for col in selected_columns if col in df_All.columns]
df_All = df_All[existing_columns]


In [7]:
# # List of columns to exclude to run XGboost feature selection
# exclude_columns = ['Octanol', 'Hexanol',
#        'Ethanol', 'Decanol',
       
#        'TI52014', 'TI55013', 'TI55014', 'TI55015', 'TI55016', 'TI55017', 'TI55021', 'TI55023',
#        'TC52015', 'FC52018', 'II52554', 'TI40050', 'VI52558B'

#        # 'FC55102', 'FC55152', 'LC55557', 'LC55568', 'TC55555',

#        # '425 SAO Al', 'FFC55553', 'LC52572', 'LC90366',

#        # 'FC42428', 'LC55553',

#        # 'FC55009'
#                    ]

# # Create a new DataFrame without the excluded columnsd
# df_All = df_All.drop(columns=exclude_columns)

In [8]:
df_All.columns

Index(['Butanol', 'FC55569', 'DI55152', '425 %Al', 'TC55552', 'FC55003',
       'LC55555', 'FFC55553', 'FFC55555', 'TC55555', 'TI55021', 'PI55004',
       'FC55552'],
      dtype='object')

In [9]:
# # Splitting into train and test
# X = df_All.drop('Butanol', axis=1)  # Assuming 'target' is your target column
# y = df_All['Butanol']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
def iterate_feature_rotations(df_all, target_column, test_size=0.2, random_state=42, num_random_iterations=30):
    results = []
    columns = [col for col in df_all.columns if col != target_column]
    random.seed(random_state)  # for reproducibility

    for feature in columns:
        for _ in range(num_random_iterations):
            # Randomly order the remaining features
            remaining_features = [f for f in columns if f != feature]
            random.shuffle(remaining_features)

            # Create a new ordered list of features
            ordered_features = [feature] + remaining_features

            reordered_df = df_all[ordered_features + [target_column]]

            # Splitting into train and test for each permutation
            X = reordered_df.drop(target_column, axis=1)
            y = reordered_df[target_column]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

            # Create and fit the XGBoost model
            model = xgb.XGBRegressor(objective='reg:squarederror')
            model.fit(X_train, y_train)

            # Extract feature importances
            feature_importances = model.get_booster().get_score(importance_type="weight")

            # Store the result with the permutation order and feature importances
            results.append((ordered_features, feature_importances))

    return results


results = iterate_feature_rotations(df_All, 'Butanol')

# Convert results to a DataFrame
flattened_results = []
for ordered_features, importances in results:
    for feature, importance in importances.items():
        flattened_results.append({
            'Feature Rotation': ordered_features,
            'Feature': feature,
            'Importance': importance
        })

results_df = pd.DataFrame(flattened_results)


In [11]:
print(results_df)

                                       Feature Rotation   Feature  Importance
0     [FC55569, TC55555, FC55003, TC55552, TI55021, ...   FC55569       739.0
1     [FC55569, TC55555, FC55003, TC55552, TI55021, ...   TC55555       442.0
2     [FC55569, TC55555, FC55003, TC55552, TI55021, ...   FC55003       405.0
3     [FC55569, TC55555, FC55003, TC55552, TI55021, ...   TC55552       499.0
4     [FC55569, TC55555, FC55003, TC55552, TI55021, ...   TI55021       371.0
...                                                 ...       ...         ...
4315  [FC55552, PI55004, TC55552, FFC55553, LC55555,...   TC55555       345.0
4316  [FC55552, PI55004, TC55552, FFC55553, LC55555,...   DI55152       507.0
4317  [FC55552, PI55004, TC55552, FFC55553, LC55555,...   FC55569       483.0
4318  [FC55552, PI55004, TC55552, FFC55553, LC55555,...  FFC55555       389.0
4319  [FC55552, PI55004, TC55552, FFC55553, LC55555,...   425 %Al       450.0

[4320 rows x 3 columns]


In [15]:
# Group by 'Feature' and calculate the average importance
average_importances = results_df.groupby('Feature')['Importance'].mean()

# Convert the Series to a DataFrame
average_importances_df = average_importances.reset_index()

# Rename the columns for clarity
average_importances_df.columns = ['Feature', 'Average Importance']

# Sort the DataFrame by 'Average Importance' in descending order
average_importances_df = average_importances_df.sort_values(by='Average Importance', ascending=False)

# Display or save the DataFrame
# print(average_importances_df)
# Or save it to a CSV file
# average_importances_df.to_csv('average_feature_importances.csv', index=False)


In [14]:
# Merge the average_importances_df with df_TagDesc
# Assuming 'ID' in df_TagDesc corresponds to 'Feature' in average_importances_df
merged_df = average_importances_df.merge(df_TagDesc, left_on='Feature', right_on='ID', how='left')

# Select only the required columns
final_df = merged_df[['Feature', 'Average Importance', 'DESCRIPTION']]

# Rename the 'DESCRIPTION' column to 'Description'
final_df.rename(columns={'DESCRIPTION': 'Description'}, inplace=True)

# Display or save the DataFrame
print(final_df)
# Or save it to a CSV file
# final_df.to_csv('average_feature_importances_with_descriptions.csv', index=False)


     Feature  Average Importance               Description
0    DI55152          548.102778    2ND STG SEPR TO FA-560
1    FC55569          541.147222    30# STM TO C4 STRIPPER
2   FFC55553          504.575000  DC551 H2O/ALKOXIDE RATIO
3    425 %Al          500.916667                       NaN
4    TC55552          486.583333  DC-551 ALKOX  FD PREHEAT
5    LC55555          473.583333   FA-552 1ST STG SEP SLRY
6    FC55552          451.541667  ALK FD TO HYDR RX DC-551
7   FFC55555          433.130556  RECYC BUT/ALKOX FD RATIO
8    PI55004          405.016667    DA-554 BOTTOM PRESSURE
9    FC55003          392.819444  DA-551 O/H H2O TO DC-551
10   TC55555          375.847222       EA-552 OUT  RECY C4
11   TI55021          374.736111            STEAM TO DA554


C:\Users\saust\AppData\Local\Temp\ipykernel_100528\3091506829.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.rename(columns={'DESCRIPTION': 'Description'}, inplace=True)
